In [1]:
import pandas as pd
import numpy as np

## Reading in the data files

In [2]:
cohort_data_file = "../data/d2_mechvent_cohort08Oct19.csv"

In [3]:
d2_cohort = pd.read_csv(cohort_data_file)
d2_cohort.shape

(20124, 47)

In [4]:
notes_file = "../data/NOTEEVENTS.csv"
notes_df = pd.read_csv(notes_file)

/home/mcb/li_lab/pnair6/venv/mcb36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print(notes_df.shape)
notes_df.head()

(2083180, 11)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [6]:
d2_cohort.head()

,ICUSTAY_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ETHNICITY,HOSPITAL_EXPIRE_FLAG,GENDER,DOD,...,LOS_ICU,ICUSTAY_SEQ,FIRST_ICU_STAY,VENTNUM,STARTTIME,ENDTIME,DURATION_HOURS,DAYS_UNTIL_DEATH,FIRST_VENT_STARTTIME,COHORT
0,211552,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,EMERGENCY,WHITE,0,M,2102-06-14,...,6.0646,1,t,1,2101-10-20 20:00:00,2101-10-24 07:44:00,83.733333,236.202778,2101-10-20 20:00:00,d2
1,220597,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,EMERGENCY,UNKNOWN/NOT SPECIFIED,1,M,2149-11-14,...,5.3231,1,t,1,2149-11-09 13:40:00,2149-11-14 12:00:00,118.333333,4.454167,2149-11-09 13:40:00,d2
2,232669,12,112213,2104-08-07 10:15:00,2104-08-20 02:57:00,ELECTIVE,WHITE,1,M,2104-08-20,...,7.6348,1,t,1,2104-08-08 04:20:00,2104-08-11 15:00:00,82.666667,12.572917,2104-08-08 04:20:00,d2
3,254478,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,EMERGENCY,WHITE,1,M,2108-08-30,...,7.9379,1,t,1,2108-08-23 15:00:00,2108-08-30 13:00:00,166.000000,7.022917,2108-08-23 15:00:00,d2
4,282039,35,166707,2122-02-10 11:15:00,2122-02-20 15:30:00,ELECTIVE,WHITE,0,F,NaN,...,5.3757,1,t,1,2122-02-10 13:00:00,2122-02-10 20:40:00,7.666667,NaN,2122-02-10 13:00:00,d2


### Number of rows which correspond to the first ICU stay of SUBJECT_ID

In [8]:
d2_cohort.FIRST_HOSP_STAY.value_counts()

t    15273
f     4851
Name: FIRST_HOSP_STAY, dtype: int64

### Retaining only those admissions in the first ICU stay

In [65]:
d2_first_icu_stay = d2_cohort[d2_cohort.FIRST_HOSP_STAY == 't']
print(d2_first_icu_stay.shape)
#d2_first_icu_stay.head()
# This result is different from what Stone gets! 

(15273, 47)


### Cleaning notes by removing NULL records of CHARTTIME and HADM_ID

In [ ]:
notes_df_filtered = notes_df[notes_df.CHARTTIME.notnull() & notes_df.HADM_ID.notnull()]

In [45]:
# Just wrote these lines of code to compare results with Stone's implementation. However results do not match!
#merge_stone = pd.merge(notes_df_filtered, d2_first_icu_stay[['HADM_ID','FIRST_VENT_STARTTIME']], on = 'HADM_ID', how='left')
#merge_stone = merge_stone[merge_stone.FIRST_VENT_STARTTIME.notnull()]
#merge_stone.drop_duplicates(subset='ROW_ID',keep='first',inplace=True)
#merge_stone.shape

#### Joining (left) the d2 cohort with the notes table on the HADM_ID
##### Removed those records whose FIRST_VENT_STARTTIME is NULL and removed duplicate  ROW_IDs

In [66]:
merged = pd.merge(d2_first_icu_stay[['HADM_ID','ICUSTAY_ID','FIRST_VENT_STARTTIME']], notes_df_filtered, on = ['HADM_ID'], how='left')
merged = merged[merged.FIRST_VENT_STARTTIME.notnull()]
merged.drop_duplicates(subset='ROW_ID',keep='first',inplace=True)
print(merged.shape)
#merged.head()

(507993, 13)


In [ ]:
#icu_charttime_diff = pd.to_timedelta(pd.to_datetime(merged_sm.ENDTIME) - pd.to_datetime(merged_sm.CHARTTIME))
#merged_sm["ICU_CHARTTIME_DIFF"] = icu_charttime_diff.astype('timedelta64[h]')
#merged_sm.head()

In [ ]:
#print(merged_sm.shape)
#merged_sm = merged_sm[merged_sm.ICU_CHARTTIME_DIFF >= 0]
#print(merged_sm.shape)

In [ ]:
#print(len(merged_sm.ICUSTAY_ID.unique()))

## Finding the time difference between ventilation start time and notes chart time. 
## This will help filter out the notes which were charted after 48 hours of admission to ventilator

In [71]:
time_diff = pd.to_timedelta(pd.to_datetime(merged.CHARTTIME) - pd.to_datetime(merged.FIRST_VENT_STARTTIME))
merged["TIME_DIFF"] = time_diff.astype('timedelta64[h]')

In [73]:
trimmed_cohort = merged[(merged.TIME_DIFF <= 48) & (merged.TIME_DIFF >= 0)]
print(trimmed_cohort.shape)
#trimmed_cohort.head()

(95412, 14)


#### Looking at the number of notes from each category

In [75]:
trimmed_cohort.CATEGORY.value_counts()

Nursing/other       37464
Radiology           27558
Nursing             14828
Physician            8968
Respiratory          4402
Nutrition            1026
General               704
Social Work           263
Rehab Services        123
Case Management        46
Consult                21
Pharmacy                9
Name: CATEGORY, dtype: int64

#### Looking at the number of unique Admission IDs

In [76]:
len(trimmed_cohort.HADM_ID.unique())

7131

#### Looking at the number of unique patient IDs

len(trimmed_cohort.SUBJECT_ID.unique())

#### Looking at the unique ICU Stay IDs

In [82]:
len(trimmed_cohort.ICUSTAY_ID.unique())

7131

In [83]:
#trimmed_cohort.to_csv("../data/d2_48hrs_Oct19.csv",index=False)

### Using ICUSTAY_ID as unit of analysis

In [78]:
trimmed_icustay_cohort = trimmed_cohort.drop_duplicates(subset="ICUSTAY_ID")
print(trimmed_icustay_cohort.shape)

(7131, 14)


#### Number of unique Admission IDs, Subject IDs and number of notes in each category

In [79]:
len(trimmed_icustay_cohort.HADM_ID.unique())

7131

In [80]:
len(trimmed_icustay_cohort.SUBJECT_ID.unique())

7131

In [81]:
trimmed_icustay_cohort.CATEGORY.value_counts()

Radiology           5317
Nursing/other        668
Nursing              557
Physician            291
Respiratory          229
General               27
Nutrition             24
Social Work           12
Rehab Services         5
Case Management        1
Name: CATEGORY, dtype: int64